## A Prompt Engineering Approach for Date Extraction Using GPT and LangChain

## Dependencies

In [ ]:
%pip install --quiet --upgrade langchain langchain-community  langchain-mistralai langchain_openai

In [ ]:
import requests
import os

from google.colab import userdata
import pandas as pd

from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import SimpleJsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

## Uploading the Original data the CSV file:

In [ ]:
df = pd.read_csv("/content/NLP_in_industry - original data .csv")

In [ ]:
text_version = df['text version'].to_list()
urls  = df['url'].to_list()

url_text_version = list(zip(urls,text_version))
url_text_version[0]

('http://www.grandchambery.fr/fileadmin/mediatheque/grand-chambery/agglomeration/organisation-politique/Bureau_communautaire/2023/Proces_verbal_du_Bureau_du_21_decembre_2023.pdf',
 'https://datapolitics-public.s3.gra.io.cloud.ovh.net/LORIA/txt/2019/c6c89e9f7afb2419432222a54525df83a756768a_Proces_verbal_du.txt')

## Download files

In [ ]:
# Dictionary to store the text content associated with each URL
texts = dict()

# Loop through each item in the 'url_text_version' list
for link in url_text_version:
    # Extract the filename from the URL (last part of the path)
    filename = link[1].split("/")[-1]

    # Send an HTTP GET request to fetch the content from the URL
    response = requests.get(link[1])

    # Check if the request was successful
    if response.status_code == 200:
        # If successful, store the response text in the dictionary with the associated key
        text = response.text
        texts[link[0]] = text
    else:
        # If the request fails, print an error message and store None for that key
        print(f"Failed to download {filename}. Status code: {response.status_code}")
        texts[link[0]] = None


Failed to download 384c7_D%C3%A9lib%C3%A9rations_Conseil_Communautaire_27_f%C3%A9vrier_2023.pdf.txt. Status code: 403
Failed to download b2cf4_CR_09_f%C3%A9vrier_2023.pdf.txt. Status code: 403
Failed to download 18c3595af8e450d0b8afffe9827a617fcfa8450f_Rapport%20de%20P.txt. Status code: 403
Failed to download 5011763f908fe9bdec498bdf9cb1517bb66fbb56_PV%20CC%20du%203.txt. Status code: 403
Failed to download 99118_PV%20int%C3%A9gral%20CM%20121222.pdf.txt. Status code: 403
Failed to download a18582e994fceea2089730c835eba47315a1cc6d_d%C3%A9lib%C3%A9.txt. Status code: 403
Failed to download 2b70d367576c4428ebddeb6d5a1b31aca597ff92_PV%20du%20CM%20d.txt. Status code: 403
Failed to download fa7ff_Liste%20d%C3%A9lib%C3%A9rations_CM%20du%2001.02.23.pdf.txt. Status code: 403
Failed to download 961d1_22-11-08-CM-Publication_D%C3%A9lib%C3%A9ration.pdf.txt. Status code: 403
Failed to download 5a70c_Sign%C3%A9_PV_CD_12%20d%C3%A9cembre%202022.pdf.txt. Status code: 403
Failed to download 44add_Proc%C3%

In [ ]:
content_df = pd.DataFrame(list(texts.items()), columns=['url', 'Content'])

## Reduce Content size to minimize the cost of tokens
the reduced content consist of 250 words from the begining and 250 words from the end

In [ ]:
def reduce_content(content: str):
    '''
    Reduces the content by keeping only the first 250 words (header) and the last 250 words (footer).

    Parameters:
    content (str): The full text content to be reduced.

    Returns:
    str: The reduced content, consisting of the header and footer, joined as a single string.
         If the input content is None, returns None.
    '''

    # Handle the case where the content is None
    if content is None:
        return None

    # Split the content into a list of words
    splitted_text = content.split()

    # Extract the first 250 words (header)
    header = splitted_text[0:500]

    # Extract the last 250 words (footer)
    footer = splitted_text[-250:]

    # Join the header and footer back into a single string and return
    return ' '.join(header + footer)

# Apply the reduce_content function to the 'Content' column
content_df["reduced_content"] = content_df["Content"].apply(reduce_content)


In [ ]:
content_df

,url,Content,reduced_content
0,http://www.grandchambery.fr/fileadmin/mediathe...,Procès-Verbal\nBureau du jeudi 21 décembre 202...,Procès-Verbal Bureau du jeudi 21 décembre 2023...
1,http://www.ville-saint-ay.fr/userfile/fichier-...,PROCES-VERBAL DE LA REUNION PUBLIQUE\nDU CONSE...,PROCES-VERBAL DE LA REUNION PUBLIQUE DU CONSEI...
2,https://www.gatine-racan.fr/wp-content/uploads...,CONSEIL COMMUNAUTAIRE DU\n25 JANVIER 2023\nPRO...,CONSEIL COMMUNAUTAIRE DU 25 JANVIER 2023 PROCE...
3,https://www.ville-mazeres.fr/IMG/pdf/2023_1_1.pdf,Date de mise en ligne de\nl’acte : 02/ 02/2023...,Date de mise en ligne de l’acte : 02/ 02/2023 ...
4,https://www.fier-et-usses.com/cms_viewFile.php...,Envoyé en préfecture le 26/01/2023\nReçu en pr...,Envoyé en préfecture le 26/01/2023 Reçu en pré...
...,...,...,...
495,https://www.estuaire-sillon.fr/fileadmin/media...,PROJET DE RAPPORT\nD’ORIENTATIONS BUDGETAIRES\...,PROJET DE RAPPORT D’ORIENTATIONS BUDGETAIRES 2...
496,https://plombieres-les-dijon.fr/wp-content/upl...,"VILLE DE PLOMBIÈRES-LÈS.DIJON\n9/f;*4^ oçAtu""r...","VILLE DE PLOMBIÈRES-LÈS.DIJON 9/f;*4^ oçAtu""rz..."
497,https://www.orne.gouv.fr/contenu/telechargemen...,Spécial n° 10 de janvier 2024\nn° 2024 01 10\n...,Spécial n° 10 de janvier 2024 n° 2024 01 10 Ma...
498,https://www.vosges.gouv.fr/contenu/telechargem...,VOSGES\n\nRECUEIL DES ACTES\nADMINISTRATIFS SP...,VOSGES RECUEIL DES ACTES ADMINISTRATIFS SPÉCIA...


In [ ]:
merged_df = pd.merge(df,content_df,on='url')[['doc_id','url','published_datapolitics','reduced_content']]

In [ ]:
merged_df

,doc_id,url,published_datapolitics,reduced_content
0,2019/c6c89e9f7afb2419432222a54525df83a756768a_...,http://www.grandchambery.fr/fileadmin/mediathe...,21/12/2023,Procès-Verbal Bureau du jeudi 21 décembre 2023...
1,6357/71845_1698228833-PV---Conseil-Municipal-1...,http://www.ville-saint-ay.fr/userfile/fichier-...,16/01/2023,PROCES-VERBAL DE LA REUNION PUBLIQUE DU CONSEI...
2,2515/213c7_proces-verbal-25-01-2023.pdf,https://www.gatine-racan.fr/wp-content/uploads...,25/01/2023,CONSEIL COMMUNAUTAIRE DU 25 JANVIER 2023 PROCE...
3,1086/ee2ec_2023_1_1.pdf,https://www.ville-mazeres.fr/IMG/pdf/2023_1_1.pdf,31/01/2023,Date de mise en ligne de l’acte : 02/ 02/2023 ...
4,3020/68132_cms_viewFile.php,https://www.fier-et-usses.com/cms_viewFile.php...,26/01/2023,Envoyé en préfecture le 26/01/2023 Reçu en pré...
...,...,...,...,...
495,2490/1bf3b3c8d457ab37c24d26b84e91ddba8673d804_...,https://www.estuaire-sillon.fr/fileadmin/media...,15/02/2024,PROJET DE RAPPORT D’ORIENTATIONS BUDGETAIRES 2...
496,6238/24d8a2f8cfd1989d316a84435f308170f1ba9fcc_...,https://plombieres-les-dijon.fr/wp-content/upl...,24/01/2024,"VILLE DE PLOMBIÈRES-LÈS.DIJON 9/f;*4^ oçAtu""rz..."
497,6812/a18ebaf196fccbea780f909e3508d7d4cb14bf6d_...,https://www.orne.gouv.fr/contenu/telechargemen...,09/01/2024,Spécial n° 10 de janvier 2024 n° 2024 01 10 Ma...
498,6834/594b09f7dd530aa0245edaf6193cb6238cd31659_...,https://www.vosges.gouv.fr/contenu/telechargem...,22/11/2022,VOSGES RECUEIL DES ACTES ADMINISTRATIFS SPÉCIA...


## Initializing the LLM:

In [ ]:
import os

# Set the OpenAI API key as an environment variable
# 'userdata.get' retrieves the API key securely from a user-defined data source
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Specify the desired model name
model_name = "gpt-4o"  # Replace "gpt-4o" with the correct model identifier if necessary

# Initialize the OpenAI chat model
llm = ChatOpenAI(model_name=model_name)


In [ ]:
def extracting_date(context:str):
  '''
  Extracting the date from a context
  input : context
  output : date
  '''

  prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that extracts the most probable 'published date' from the given text in DD/MM/YYYY format. If no date is found, return 'None'.\
             Output the result as a JSON with a single key 'date' and the corresponding value.",),
          ("human", "{input}"),
    ]
  )
  json_parser = SimpleJsonOutputParser()
  chain = prompt | llm | json_parser
  result = chain.invoke(
    {
        "input": context,
    }
  )
  return result.get("date")


In [ ]:
merged_df

,doc_id,url,published_datapolitics,reduced_content
0,2019/c6c89e9f7afb2419432222a54525df83a756768a_...,http://www.grandchambery.fr/fileadmin/mediathe...,21/12/2023,Procès-Verbal Bureau du jeudi 21 décembre 2023...
1,6357/71845_1698228833-PV---Conseil-Municipal-1...,http://www.ville-saint-ay.fr/userfile/fichier-...,16/01/2023,PROCES-VERBAL DE LA REUNION PUBLIQUE DU CONSEI...
2,2515/213c7_proces-verbal-25-01-2023.pdf,https://www.gatine-racan.fr/wp-content/uploads...,25/01/2023,CONSEIL COMMUNAUTAIRE DU 25 JANVIER 2023 PROCE...
3,1086/ee2ec_2023_1_1.pdf,https://www.ville-mazeres.fr/IMG/pdf/2023_1_1.pdf,31/01/2023,Date de mise en ligne de l’acte : 02/ 02/2023 ...
4,3020/68132_cms_viewFile.php,https://www.fier-et-usses.com/cms_viewFile.php...,26/01/2023,Envoyé en préfecture le 26/01/2023 Reçu en pré...
...,...,...,...,...
495,2490/1bf3b3c8d457ab37c24d26b84e91ddba8673d804_...,https://www.estuaire-sillon.fr/fileadmin/media...,15/02/2024,PROJET DE RAPPORT D’ORIENTATIONS BUDGETAIRES 2...
496,6238/24d8a2f8cfd1989d316a84435f308170f1ba9fcc_...,https://plombieres-les-dijon.fr/wp-content/upl...,24/01/2024,"VILLE DE PLOMBIÈRES-LÈS.DIJON 9/f;*4^ oçAtu""rz..."
497,6812/a18ebaf196fccbea780f909e3508d7d4cb14bf6d_...,https://www.orne.gouv.fr/contenu/telechargemen...,09/01/2024,Spécial n° 10 de janvier 2024 n° 2024 01 10 Ma...
498,6834/594b09f7dd530aa0245edaf6193cb6238cd31659_...,https://www.vosges.gouv.fr/contenu/telechargem...,22/11/2022,VOSGES RECUEIL DES ACTES ADMINISTRATIFS SPÉCIA...


In [ ]:
merged_df['date_from_url'] = merged_df['url'].apply(extracting_date)

LLM couldn't extract the most probable "published date" from 164 examples

In [ ]:
merged_df['date_from_url'].value_counts(dropna=False)

,count
date_from_url,
None,164
16/02/2023,6
12/12/2022,6
13/02/2023,5
15/12/2022,5
...,...
10/05/2022,1
30/11/2022,1
02/11/2010,1


## Applying the extracting_date function on the reduced_text

In [ ]:
merged_df['date_from_reduced_content'] = merged_df.query('date_from_url == "None"')['reduced_content'].apply(extracting_date)

In [ ]:
print(merged_df[['date_from_reduced_content','date_from_url']])

    date_from_reduced_content date_from_url
0                         NaN    21/12/2023
1                         NaN    16/01/2023
2                         NaN    25/01/2023
3                         NaN    01/01/2023
4                  26/01/2023          None
..                        ...           ...
495                15/02/2024          None
496                       NaN    24/01/2024
497                       NaN    09/01/2024
498                22/11/2022          None
499                       NaN    22/12/2023

[500 rows x 2 columns]


In [ ]:
merged_df['final_date'] = merged_df.apply(
    lambda row: row['date_from_reduced_content'] if not pd.isna(row['date_from_reduced_content']) else row['date_from_url'],
    axis=1
)

In [ ]:
merged_df[['url','final_date']]['final_date'].value_counts(dropna=False)

,count
final_date,
None,31
13/12/2022,10
28/06/2022,7
16/02/2023,7
19/05/2022,7
...,...
30/11/2023,1
29/06/2022,1
30/11/2022,1


## Evaluation:

In [ ]:
import pandas as pd

# Load the Gold standard dataset containing publication dates
gold_df = pd.read_csv("hf://datasets/maribr/publication_dates_fr/NLP_in_industry - Annotations (3).csv")

# Merge the DataFrame containing 'url' and 'final_date' with the Gold dataset
# on the 'url' column to align publication dates
final_df = pd.merge(
    merged_df[['url', 'final_date']],  # Select only the relevant columns from merged_df
    gold_df,                          # The Gold dataset with 'Gold published date'
    on='url'                          # Merge on the 'url' column
)[['url', 'Gold published date', 'final_date']]  # Retain only the necessary columns

# Calculate the accuracy by comparing 'final_date' with 'Gold published date'
# Compute the proportion of exact matches
accuracy = (final_df['final_date'] == final_df['Gold published date']).mean()

# Print the accuracy as a percentage with two decimal places
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 64.60%
